In [1]:
import requests

In [2]:
from openai import OpenAI

In [3]:
import json

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
import os

In [7]:
stock_api = os.getenv("ALPHA_VANTAGE_API_KEY")

In [8]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [51]:
from agents import Agent, Runner, function_tool, ModelSettings, SQLiteSession, trace, RunConfig

In [10]:
from functools import lru_cache

In [11]:
from datetime import datetime, timedelta

In [12]:
url = "https://www.alphavantage.co/query?"

In [13]:
import json

In [14]:
@function_tool
def find_ticker(text: str) -> str:
    """Based on the natural language query, use this function to find the ticker symbol"""

    # params = {
    #     "function": "SYMBOL_SEARCH",
    #     "keywords": text,
    #     "apikey": stock_api
    # }

    # response = requests.get(url, params)

    # data = response.json()

    # ticker = (data.get('bestMatches')[0]).get('1. symbol')

    return _find_ticker(text)

In [15]:
@lru_cache(maxsize = 100)
def _find_ticker(text: str) -> str:
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": text,
        "apikey": stock_api
    }

    response = requests.get(url, params)

    data = response.json()

    ticker = data.get('bestMatches')

    filter_region = []

    for t in ticker:
        if t.get('4. region') == 'United States':
            filter_region.append(t)

    return filter_region[0].get('1. symbol')

In [16]:
_find_ticker('Microsoft')

'MSFT'

In [16]:
@function_tool
def get_income_statement(ticker: str) -> str:
    """Based on the ticker you can find the income statement of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    return _get_income_statement(ticker)

In [17]:
@lru_cache(maxsize = 100)
def _get_income_statement(ticker: str) -> str:
    """Based on the ticker you can find the income statement of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": stock_api
    }

    income_statement = requests.get(url,params)

    income_statement_data = income_statement.json()

    annual_data = income_statement_data.get('annualReports')

    quarterly_data = income_statement_data.get('quarterlyReports')

    filtered_data_annual = []

    filtered_data_quarterly = []

    for report in annual_data:
        filtered_data_annual.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })

    for report_quarterly in quarterly_data:
        filtered_data_quarterly.append({
            'fiscalDateEnding': report_quarterly.get('fiscalDateEnding'),
            'grossProfit': report_quarterly.get('grossProfit'),
            'totalRevenue': report_quarterly.get('totalRevenue'),
            'costOfRevenue': report_quarterly.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report_quarterly.get('costofGoodsAndServicesSold'),
            'operatingIncome': report_quarterly.get('operatingIncome'),
            'operatingExpenses': report_quarterly.get('operatingExpenses'),
            'researchAndDevelopment': report_quarterly.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report_quarterly.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report_quarterly.get('netInterestIncome'),
            'interestIncome': report_quarterly.get('interestIncome'),
            'depreciationAndAmortization': report_quarterly.get('depreciationAndAmortization'),
            'incomeBeforeTax': report_quarterly.get('incomeBeforeTax'),
            'incomeTaxExpense': report_quarterly.get('incomeTaxExpense'),
            'ebit': report_quarterly.get('ebit'),
            'ebitda': report_quarterly.get('ebitda'),
            'netIncome': report_quarterly.get('netIncome'),
        })

    final_data = {
        "annual_data": filtered_data_annual,
        "quarterly_data": filtered_data_quarterly[:8]
    }

    return final_data

In [206]:
_get_income_statement('TSLA')

{'annual_data': [{'fiscalDateEnding': '2024-12-31',
   'grossProfit': '17450000000',
   'totalRevenue': '97690000000',
   'costOfRevenue': '80240000000',
   'costofGoodsAndServicesSold': '80240000000',
   'operatingIncome': '7076000000',
   'operatingExpenses': '10374000000',
   'researchAndDevelopment': '4540000000',
   'sellingGeneralAndAdministrative': '5150000000',
   'netInterestIncome': '1219000000',
   'interestIncome': '1569000000',
   'depreciationAndAmortization': '5368000000',
   'incomeBeforeTax': '8990000000',
   'incomeTaxExpense': '1837000000',
   'ebit': '9340000000',
   'ebitda': '14708000000',
   'netIncome': '7130000000'},
  {'fiscalDateEnding': '2023-12-31',
   'grossProfit': '17660000000',
   'totalRevenue': '96773000000',
   'costOfRevenue': '79113000000',
   'costofGoodsAndServicesSold': '79113000000',
   'operatingIncome': '8891000000',
   'operatingExpenses': '8769000000',
   'researchAndDevelopment': '3969000000',
   'sellingGeneralAndAdministrative': '4800000

In [18]:
@function_tool
def get_annual_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement.
    """

    annual_filter = json_file.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })

    return filtered_data

In [101]:
get_annual_income_statement(cached_income_statement)

[{'fiscalDateEnding': '2024-12-31',
  'grossProfit': '17450000000',
  'totalRevenue': '97690000000',
  'costOfRevenue': '80240000000',
  'costofGoodsAndServicesSold': '80240000000',
  'operatingIncome': '10374000000',
  'researchAndDevelopment': '4540000000',
  'sellingGeneralAndAdministrative': '5150000000',
  'netInterestIncome': '1219000000',
  'interestIncome': '1569000000',
  'depreciationAndAmortization': '5368000000',
  'incomeBeforeTax': '8990000000',
  'incomeTaxExpense': '1837000000',
  'ebit': '9340000000',
  'ebitda': '14708000000',
  'netIncome': '7130000000'},
 {'fiscalDateEnding': '2023-12-31',
  'grossProfit': '17660000000',
  'totalRevenue': '96773000000',
  'costOfRevenue': '79113000000',
  'costofGoodsAndServicesSold': '79113000000',
  'operatingIncome': '8769000000',
  'researchAndDevelopment': '3969000000',
  'sellingGeneralAndAdministrative': '4800000000',
  'netInterestIncome': '910000000',
  'interestIncome': '1066000000',
  'depreciationAndAmortization': '46670

In [19]:
@function_tool
def get_quarterly_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement datapoints.
    """

    quarterly_filter = json_file.get('quarterlyReports')

    filtered_qoq_data = []

    for report in quarterly_filter:
        filtered_qoq_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })


    return filtered_qoq_data[:4]

In [20]:
@function_tool
def get_balance_sheet(ticker: str) -> str:
    """Based on the ticker you can find the balance sheet of the company from this
    function. This will fetch both annual balance sheet only.
    """

    return _get_balance_sheet(ticker)

In [21]:
@lru_cache
def _get_balance_sheet(ticker: str) -> str:

    params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": stock_api
    }

    balance_sheet = requests.get(url,params)

    balance_sheet_data = balance_sheet.json()

    annual_data = balance_sheet_data.get('annualReports')

    quarterly_data = balance_sheet_data.get('quarterlyReports')

    filtered_data_annual = []

    filtered_data_quarterly = []

    for report in annual_data:
        filtered_data_annual.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
        })

    for report_quarterly in quarterly_data:
        filtered_data_quarterly.append({
            'fiscalDateEnding': report_quarterly.get('fiscalDateEnding'),
            'totalAssets': report_quarterly.get('totalAssets'),
            'totalCurrentAssets': report_quarterly.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report_quarterly.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report_quarterly.get('inventory'),
            'totalNonCurrentAssets': report_quarterly.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report_quarterly.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report_quarterly.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report_quarterly.get('totalLiabilities'),
            'interestIncome': report_quarterly.get('interestIncome'),
            'totalCurrentLiabilities': report_quarterly.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report_quarterly.get('currentLongTermDebt'),
            'totalShareholderEquity': report_quarterly.get('totalShareholderEquity'),
            'retainedEarnings': report_quarterly.get('retainedEarnings'),
        })


    final_data = {
        "annual_data": filtered_data_annual,
        "quarterly_data": filtered_data_quarterly[:8]
    }

    

    return final_data

In [201]:
_get_balance_sheet('TSLA')

{'annual_data': [{'fiscalDateEnding': '2024-12-31',
   'totalAssets': '122070000000',
   'totalCurrentAssets': '58360000000',
   'cashAndCashEquivalentsAtCarryingValue': '16139000000',
   'inventory': '12017000000',
   'totalNonCurrentAssets': '63716000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'sellingGeneralAndAdministrative': None,
   'totalLiabilities': '48390000000',
   'interestIncome': None,
   'totalCurrentLiabilities': '28821000000',
   'currentLongTermDebt': '2343000000',
   'totalShareholderEquity': '72913000000',
   'retainedEarnings': '35209000000'},
  {'fiscalDateEnding': '2023-12-31',
   'totalAssets': '106618000000',
   'totalCurrentAssets': '49616000000',
   'cashAndCashEquivalentsAtCarryingValue': '16398000000',
   'inventory': '13626000000',
   'totalNonCurrentAssets': '57003000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'sellingGeneralAndAdministrative': None,
   'totalLiabilities': '43009000000',
   'interestIncome': None,
   't

In [119]:
cached_balance_sheet = get_balance_sheet(ticker_symbol)

cached_balance_sheet

{'symbol': 'TSLA',
 'annualReports': [{'fiscalDateEnding': '2024-12-31',
   'reportedCurrency': 'USD',
   'totalAssets': '122070000000',
   'totalCurrentAssets': '58360000000',
   'cashAndCashEquivalentsAtCarryingValue': '16139000000',
   'cashAndShortTermInvestments': '16139000000',
   'inventory': '12017000000',
   'currentNetReceivables': '4418000000',
   'totalNonCurrentAssets': '63716000000',
   'propertyPlantEquipment': '51501000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'intangibleAssets': '1226000000',
   'intangibleAssetsExcludingGoodwill': '1226000000',
   'goodwill': '244000000',
   'investments': 'None',
   'longTermInvestments': 'None',
   'shortTermInvestments': '20424000000',
   'otherCurrentAssets': '5362000000',
   'otherNonCurrentAssets': 'None',
   'totalLiabilities': '48390000000',
   'totalCurrentLiabilities': '28821000000',
   'currentAccountsPayable': '12474000000',
   'deferredRevenue': 'None',
   'currentDebt': 'None',
   'shortTermDebt': '32

In [22]:
@function_tool
def annual_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding annual balance sheet datapoints.
    """

    annual_filter = json_data.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data

In [23]:
@function_tool
def quarterly_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding quarterly balance sheet datapoints.
    """

    quarterly_filter = json_data.get('quarterlyReports')

    filtered_data = []

    for report in quarterly_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data

In [24]:
@function_tool
def cash_flow_statement(ticker: str) -> str:
    """Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual cash flow statement.
    """

    return _cash_flow_statement(ticker)

In [25]:
@lru_cache
def _cash_flow_statement(ticker: str) -> str:
    """Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual cash flow statement.
    """

    params = {
        "function": "CASH_FLOW",
        "symbol": ticker,
        "apikey": stock_api
    }

    cash_flow = requests.get(url,params)

    cash_flow_data = cash_flow.json()

    annual_filter = cash_flow_data.get('annualReports')

    final_cash_flow_data = []

    for cash_flow in annual_filter:
        final_cash_flow_data.append({
            'fiscalDateEnding': cash_flow.get('fiscalDateEnding'),
            'operatingCashflow': cash_flow.get('operatingCashflow'),
            'cashflowFromInvestment': cash_flow.get('cashflowFromInvestment'),
            'cashflowFromFinancing': cash_flow.get('cashflowFromFinancing'),
            'netIncome': cash_flow.get('netIncome')
        })

    return final_cash_flow_data

In [134]:
cash_flow_statement(ticker_symbol)

[{'fiscalDateEnding': '2024-12-31',
  'operatingCashflow': '14923000000',
  'cashflowFromInvestment': '-18787000000',
  'cashflowFromFinancing': '3853000000',
  'netIncome': '7153000000'},
 {'fiscalDateEnding': '2023-12-31',
  'operatingCashflow': '13256000000',
  'cashflowFromInvestment': '-15584000000',
  'cashflowFromFinancing': '2589000000',
  'netIncome': '14974000000'},
 {'fiscalDateEnding': '2022-12-31',
  'operatingCashflow': '14724000000',
  'cashflowFromInvestment': '-11973000000',
  'cashflowFromFinancing': '-3527000000',
  'netIncome': '12556000000'},
 {'fiscalDateEnding': '2021-12-31',
  'operatingCashflow': '11497000000',
  'cashflowFromInvestment': '-7868000000',
  'cashflowFromFinancing': '-5203000000',
  'netIncome': '5519000000'},
 {'fiscalDateEnding': '2020-12-31',
  'operatingCashflow': '5943000000',
  'cashflowFromInvestment': '-3132000000',
  'cashflowFromFinancing': '9973000000',
  'netIncome': '690000000'},
 {'fiscalDateEnding': '2019-12-31',
  'operatingCashflo

In [26]:
@function_tool
def price_data(ticker: str) -> str:
    """ 
    Based on the ticker, we need to find the price of the stock through this.
    """

    return _price_data(ticker)

In [27]:
@lru_cache
def _price_data(ticker: str) -> str:
    """ 
    Based on the ticker, we need to find the price of the stock through this.
    """

    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": "compact",
        "apikey": stock_api
    }

    response = requests.get(url,params)

    response_json = response.json()

    filtered_json = response_json['Time Series (Daily)']

    close_price = []

    for date, values in filtered_json.items():
        close_price.append(
            {
                'date': date,
                'close': values['4. close']
            }
        )

    latest_price = close_price[0]

    return latest_price

In [28]:
@function_tool
def earnings_analysis(ticker: str) -> str:
    """ 
    Based on the ticker, you will find a 10-12 point analysis related to the latest earnings report. 
    You can use it in case the user wants to do some kind of qualitative analysis
    """

    return _earnings_analysis(ticker)

In [29]:
@lru_cache
def _earnings_analysis(ticker: str) -> str:
    """
    Based on the ticker, you will find a 10-12 point analysis related to the latest earnings report. 
    You can use it in case the user wants to do some kind of qualitative analysis
    """

    final_transcript = []

    final_quarter = []

    while final_transcript == []:
        today = datetime.now()
    
        year = today.year
        month = today.month
    
        if month in [1,2,3]:
            quarter = 'Q4'
            year -= 1
        elif month in [4,5,6]:
            quarter = 'Q1'
        elif month in [7,8,9]:
            quarter = 'Q2'
        else:
            quarter = 'Q3'
    
        final = f'{year}{quarter}'
    
        params = {
            "function": "EARNINGS_CALL_TRANSCRIPT",
            "symbol": ticker,
            "apikey": stock_api,
            "quarter": final
        }

        response = requests.get(url, params)

        final_json = response.json()

        transcript = final_json.get('transcript')

        quarter = final_json.get('quarter')

        final_transcript.append(transcript)

        final_quarter.append(quarter)

    
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": f""" 
           You will be given the full earnings call transcript in {final_transcript}.

            Your task is to extract **7–10 investor-relevant insights** strictly based on what 
            **Apple management said** in the transcript.
            
            Rules you MUST follow:
            1. Use ONLY information explicitly stated by Apple executives (CEO, CFO, or Investor Relations).  
               Do NOT include analyst opinions, assumptions, or external interpretation.
            2. Each point must represent a **distinct insight**, not a restatement of another point.
            3. For every point, clearly mention **who said it** (name + role).
            4. Do NOT use generic phrases such as “strong performance”, “positive outlook”, or “solid results” 
            unless backed by specific data or commentary.
            5. Each point must explain **why this matters to a long-term investor** 
            (growth, margins, cash flow, competitive position, or strategic direction).
            6. Prefer **synthesis over repetition** — combine numbers with management commentary where relevant.
            7. Do NOT introduce any facts, estimates, or forward views not explicitly stated in the transcript.
            
            Output format:
            - Heading: Analysis of the earnings call ({final_quarter})
            - Numbered list (7–10 points)
            - 2–4 sentences per point
            - End each point with: *(Source: Name, Title)*
            
            Tone:
            - Analytical
            - Investor-oriented
            - Neutral and factual

            Don't only include positive or negative points, try to include both
            """}
        ], temperature = 0.3
    )

    return response.choices[0].message.content

In [172]:
_earnings_analysis('MSFT')

"# Analysis of the earnings call (['2025Q3'])\n\n1. Microsoft Cloud revenue surpassed $42 billion, reflecting a 22% growth in constant currency. This significant increase underscores the company's strong position in the cloud market, which is critical for long-term growth as businesses increasingly migrate to cloud solutions. The robust demand for cloud services indicates that Microsoft is effectively capturing market share and expanding its customer base. *(Source: Satya Nadella, CEO)*\n\n2. The company reported a commercial bookings increase of 18%, driven by Azure commitments from major clients like OpenAI. This growth in bookings is a positive indicator of future revenue, as it suggests that customers are making long-term commitments to Microsoft's cloud services, which can enhance revenue predictability and stability. *(Source: Amy Hood, CFO)*\n\n3. Microsoft’s commercial remaining performance obligation rose to $315 billion, up 34% year-over-year. This metric is crucial as it ref

In [45]:
# global_session = SQLiteSession("test_conversation")

In [60]:
async def _agent_builder(query, workflow = "random_test"):

    agent = Agent(
        name = "Data Finder",
        model = 'gpt-4o-mini',
        instructions = f"""
        You are a stock analyser and your job is to understand the {query} and then use the tools available at your disposal 
        to answer to the user. 

        Make sure that you ONLY get data from tools and in case the query cannot be solved by tools, empathetically refuse to answer
        the user query. 

        It is recommended you use tables, if you're doing any comparison either between 2 or more companies or you're 
        showing the data of more than 1 year.

        In the tools you are now getting annual and quarterly information for {get_income_statement} & {get_balance_sheet}.

        You can use both to do your analysis now. 
        
        Quarterly data is for the last 8 quarters!

        You can use the {earnings_analysis} tool in case the user wants to know what happened in the latest earnings call of a particular
        company.

        However, you should always try to back it up with financial data first. 
        
        EXCEPTION:
        If the user is specifically asking to summarise the earning call data you are supposed to simply just print out
        the output you're getting from {earnings_analysis}.
        """,
        tools = [
            price_data,
            cash_flow_statement,
            find_ticker,
            get_balance_sheet,
            get_income_statement,
            earnings_analysis
        ],
        model_settings = ModelSettings(
            tool_choice = "auto",
            seed = 0,
            temperature = 0.3
        )
    )
    
    result = await Runner.run(agent, query, run_config = RunConfig(workflow_name = workflow))

    return print(result.final_output)

    # tools_called = {
    #     item.raw_item.name
    #     for item in result.new_items

    #     if item.type == "tool_call_item"
    # }
    # return tools_called

In [61]:
await _agent_builder("Does google have a greater revenue than microsoft?", workflow = "Income Statement Check")

To determine whether Google (Alphabet Inc.) has greater revenue than Microsoft, let's compare their total revenue for the most recent fiscal year available.

### Annual Revenue Comparison

| Company       | Fiscal Year Ending | Total Revenue (in billions) |
|---------------|--------------------|-----------------------------|
| Alphabet Inc. | 2023-12-31         | $307.39                     |
| Microsoft     | 2023-06-30         | $211.92                     |

### Conclusion

Based on the latest available data, **Alphabet Inc. (Google)** has greater revenue than **Microsoft** for the most recent fiscal year.


In [62]:
await _agent_builder("Compare QoQ revenue of Tesla and see whether it is growing or not?", workflow = "Income Statement Check")

Here's the comparison of Tesla's quarterly revenue over the last two quarters to determine if it is growing:

| Quarter Ending   | Total Revenue (in billions) | QoQ Change (in billions) |
|------------------|-----------------------------|---------------------------|
| 2023-12-31       | 25.167                      | -                         |
| 2024-03-31       | 21.301                      | -3.866                    |
| 2024-06-30       | 25.500                      | +4.199                    |
| 2024-09-30       | 28.095                      | +2.595                    |

### Analysis:
- **From 2023-12-31 to 2024-03-31**: Revenue decreased by **$3.866 billion**.
- **From 2024-03-31 to 2024-06-30**: Revenue increased by **$4.199 billion**.
- **From 2024-06-30 to 2024-09-30**: Revenue further increased by **$2.595 billion**.

### Conclusion:
Tesla's revenue showed a decline in the first quarter of 2024 but has since rebounded with growth in the subsequent quarters. Overall, the tren

In [63]:
await _agent_builder("What is the breakdown of cost of Tesla, give an analysis of the same", workflow = "Income Statement Check")

### Breakdown of Costs for Tesla (TSLA)

#### Annual Income Statement Overview

| Fiscal Year Ending | Total Revenue | Cost of Revenue | Gross Profit | Operating Expenses | Net Income |
|--------------------|---------------|------------------|--------------|--------------------|------------|
| 2024-12-31         | $97,690,000,000 | $80,240,000,000  | $17,450,000,000 | $10,374,000,000    | $7,130,000,000 |
| 2023-12-31         | $96,773,000,000 | $79,113,000,000  | $17,660,000,000 | $8,769,000,000     | $14,974,000,000 |
| 2022-12-31         | $81,462,000,000 | $60,609,000,000  | $20,853,000,000 | $7,021,000,000     | $12,587,000,000 |
| 2021-12-31         | $53,823,000,000 | $40,217,000,000  | $13,606,000,000 | $7,110,000,000     | $5,644,000,000 |
| 2020-12-31         | $31,536,000,000 | $24,906,000,000  | $6,630,000,000  | $4,636,000,000     | $721,000,000  |

#### Cost Breakdown Analysis

1. **Cost of Revenue**: This is the largest expense for Tesla, consistently accounting for appr

In [64]:
await _agent_builder("Could you tell me about the cost of revenue for Tesla?", workflow = "Income Statement Check")

Here's the cost of revenue for Tesla (TSLA) over the past few years, along with the total revenue for context:

| Fiscal Year Ending | Total Revenue (USD) | Cost of Revenue (USD) |
|--------------------|---------------------|------------------------|
| 2024-12-31         | 97,690,000,000      | 80,240,000,000         |
| 2023-12-31         | 96,773,000,000      | 79,113,000,000         |
| 2022-12-31         | 81,462,000,000      | 60,609,000,000         |
| 2021-12-31         | 53,823,000,000      | 40,217,000,000         |
| 2020-12-31         | 31,536,000,000      | 24,906,000,000         |
| 2019-12-31         | 24,578,000,000      | 20,509,000,000         |
| 2018-12-31         | 21,461,268,000      | 17,419,247,000         |
| 2017-12-31         | 11,759,000,000      | 9,536,000,000         |

### Summary:
- The cost of revenue has increased significantly over the years, reflecting Tesla's growth in production and sales.
- In 2024, the cost of revenue is projected to be approxima

In [65]:
await _agent_builder("Is tesla growing year on year and quarter on quarter?", workflow = "Income Statement Check")

To determine if Tesla (TSLA) is growing year-on-year (YoY) and quarter-on-quarter (QoQ), we can analyze the financial data from the latest income statements and balance sheets.

### Year-on-Year Analysis

#### Annual Income Statement Summary

| Fiscal Year Ending | Total Revenue | Gross Profit | Net Income |
|---------------------|---------------|--------------|------------|
| 2024-12-31          | $97.69B       | $17.45B      | $7.13B     |
| 2023-12-31          | $96.77B       | $17.66B      | $14.97B    |
| 2022-12-31          | $81.46B       | $20.85B      | $12.59B    |
| 2021-12-31          | $53.82B       | $13.61B      | $5.64B     |

#### Year-on-Year Growth Rates

- **Total Revenue Growth**:
  - 2024 vs 2023: \( \frac{97.69 - 96.77}{96.77} \times 100 \approx 0.95\% \)
  - 2023 vs 2022: \( \frac{96.77 - 81.46}{81.46} \times 100 \approx 18.79\% \)

- **Net Income Growth**:
  - 2024 vs 2023: \( \frac{7.13 - 14.97}{14.97} \times 100 \approx -52.45\% \)
  - 2023 vs 2022: \( \frac{

In [66]:
# Test Results:
# 1. 40% pass rate
# 2. Agent is calling balance sheet, even for basic queries that we are asking

In [68]:
await _agent_builder("Give me a breakdown of Tesla's assets & liabilities?", workflow = "Balance Sheet Check")

Here's a breakdown of Tesla's assets and liabilities based on the most recent annual data:

### Annual Balance Sheet Summary

| Fiscal Year Ending | Total Assets | Total Current Assets | Cash & Cash Equivalents | Inventory | Total Non-Current Assets | Total Liabilities | Total Current Liabilities | Current Long-Term Debt | Total Shareholder Equity | Retained Earnings |
|---------------------|--------------|----------------------|-------------------------|-----------|--------------------------|-------------------|--------------------------|-----------------------|-------------------------|-------------------|
| 2024-12-31          | $122,070,000,000 | $58,360,000,000     | $16,139,000,000        | $12,017,000,000 | $63,716,000,000         | $48,390,000,000   | $28,821,000,000         | $2,343,000,000       | $72,913,000,000       | $35,209,000,000   |
| 2023-12-31          | $106,618,000,000 | $49,616,000,000     | $16,398,000,000        | $13,626,000,000 | $57,003,000,000         | $43

In [69]:
await _agent_builder("Is Tesla's assets increasing YoY & QoQ?", workflow = "Balance Sheet Check")

To determine if Tesla's assets are increasing Year-over-Year (YoY) and Quarter-over-Quarter (QoQ), let's analyze the data from the balance sheets.

### Annual Data Comparison
| Fiscal Year Ending | Total Assets (in billions) | YoY Change (in billions) |
|---------------------|---------------------------|--------------------------|
| 2024-12-31          | 122.07                    | +15.45                   |
| 2023-12-31          | 106.62                    | +24.24                   |
| 2022-12-31          | 82.34                     | +19.02                   |
| 2021-12-31          | 62.13                     | +5.65                    |
| 2020-12-31          | 52.15                     | +8.06                    |

### Quarterly Data Comparison
| Fiscal Quarter Ending | Total Assets (in billions) | QoQ Change (in billions) |
|-----------------------|---------------------------|--------------------------|
| 2025-09-30            | 133.74                    | +5.17                   

In [70]:
await _agent_builder("Compare the inventory of tesla with its current assets?", workflow = "Balance Sheet Check")

[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: max retries reached, giving up on this batch.


Here's a comparison of Tesla's inventory with its current assets based on the latest available data:

### Tesla's Inventory vs. Current Assets

| Fiscal Year Ending | Inventory (in $) | Total Current Assets (in $) |
|--------------------|------------------|-----------------------------|
| 2024-12-31         | 12,017,000,000   | 58,360,000,000              |
| 2023-12-31         | 13,626,000,000   | 49,616,000,000              |
| 2022-12-31         | 12,839,000,000   | 40,917,000,000              |
| 2021-12-31         | 5,757,000,000    | 27,100,000,000              |
| 2020-12-31         | 4,101,000,000    | 26,717,000,000              |
| 2019-12-31         | 3,552,000,000    | 12,103,000,000              |

### Analysis
- **Current Assets**: This includes cash, inventory, receivables, and other short-term assets. 
- **Inventory**: Represents the goods available for sale, which is a part of the current assets.

### Key Observations
1. **Inventory Growth**: Tesla's inventory has gene

In [71]:
await _agent_builder("Are shareholders equity rising for Tesla?", workflow = "Balance Sheet Check")

To determine if Tesla's shareholders' equity is rising, let's examine the annual data for the last few years:

| Fiscal Year Ending | Total Shareholder Equity (in billions) |
|--------------------|----------------------------------------|
| 2024-12-31         | 72.91                                  |
| 2023-12-31         | 62.63                                  |
| 2022-12-31         | 44.70                                  |
| 2021-12-31         | 30.19                                  |
| 2020-12-31         | 22.23                                  |

### Analysis
- **2024**: $72.91 billion
- **2023**: $62.63 billion
- **2022**: $44.70 billion
- **2021**: $30.19 billion
- **2020**: $22.23 billion

### Conclusion
Tesla's total shareholders' equity has been consistently rising over the past few years. The increase from $62.63 billion in 2023 to $72.91 billion in 2024 indicates a positive trend in shareholders' equity.


In [72]:
await _agent_builder("Compare non current assets with current assets for tesla", workflow = "Balance Sheet Check")

Here's a comparison of Tesla's non-current assets and current assets over the last few fiscal years:

| Fiscal Year Ending | Total Current Assets (USD) | Total Non-Current Assets (USD) |
|--------------------|----------------------------|---------------------------------|
| 2024-12-31         | 58,360,000,000             | 63,716,000,000                  |
| 2023-12-31         | 49,616,000,000             | 57,003,000,000                  |
| 2022-12-31         | 40,917,000,000             | 41,421,000,000                  |
| 2021-12-31         | 27,100,000,000             | 35,031,000,000                  |
| 2020-12-31         | 26,717,000,000             | 25,434,000,000                  |
| 2019-12-31         | 12,103,000,000             | 22,203,000,000                  |
| 2018-12-31         | 8,307,000,000              | 21,435,000,000                  |
| 2017-12-31         | 6,570,520,000              | 22,084,948,000                  |

### Key Observations:
- **Growth in As

In [ ]:
#For balance sheet related queries: 
#We have received a 100% pass rate, the agent is calling either find ticker or get balance sheet or both, and is not calling anything else